In [34]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import numba

In [41]:
L = 250
N = 100
x = np.arange(-int(L/2), int(L/2), 1)
y = np.arange(-int(L/2), int(L/2), 1)
sp = np.meshgrid(x, y)
v = 0.03


##x represents coordinates, v has the components of the velocity (probably superfluous, psi is the orientation, omega is the 
##angular velocity and r is the radius.
particles_x = np.zeros([N, 2])
particles_v = np.zeros([N, 2])
particles_psi = np.zeros(N)
particles_omega = np.zeros(N)
particles_r = np.zeros(N)

In [44]:
for i in range (int(np.sqrt(N))):
    particles_x[10*i:10*(i+1), 0] = np.arange(0, 10)
    particles_x[10*i:10*(i+1), 1] = i
    
particles_r[:] = np.random.normal(1, 0.1, N)
particles_psi = particles_psi - d_theta
particles_v[:, 0] = v*np.cos(particles_psi)
particles_v[:, 0] = v*np.sin(particles_psi)

In [70]:
def update(N, particles_r, particles_psi):
    F_x = np.zeros(N)
    F_y = np.zeros(N)
    theta_out = np.zeros(N)
    d_theta = np.zeros(N)
    theta_noise = np.random.random(N)
    dx = np.zeros(N)
    dy = np.zeros(N)
    v = 0.3
    particles_omega = v/particles_r
    
    ##distances and angles calculations
    d = np.linalg.norm(particles_x[None, :, :] - particles_x[:, None, :], axis = 2)
    d[d > 2.7] = 0
    select_matrix = np.copy(d)
    select_matrix[select_matrix > 0] = 1
    angles = particles_x[None, :, :] - particles_x[:, None, :]
    angles.diag[i, i, :] = 100
    
    ##normally, atan return a value between -pi/2 and pi/2. atan2 returns a value between pi and -pi.
    angles = select_matrix*np.atan2(particles_x[:, :, 1]/particles_x[:, :, 0])
    
    ##distances along different axes
    dx = d*np.cos(angles[:])
    dy = d*np.sin(angles[:])
    
    ##NEEDS CHECKING. Pretty sure it's wrong.
    if (max(abs(angles[i, :])) - min(abs(angles[i, :])) - np.pi < 0):
        d_theta = max(angles[i, :]) - min(angles[i, :])
        theta_out[:] = 2*np.pi - d_theta
    
    ##force calculations
    F_x[:] = (v + (d_theta[:] - np.pi)*d_theta[:]*np.piecewise(d_theta[:] -np.pi > 0))*np.cos(particles_psi[:]) - k*np.sum(dx, axis = 1)[:]
    F_y[:] = (v + (d_theta[:] - np.pi)*d_theta[:]*np.piecewise(d_theta[:] -np.pi > 0))*np.sin(particles_psi[:]) - k * np.sum(dy, axis = 1)[:]
    
    T = particles:omega[:]*theta_noise*np.piecewise(d_theta[:] -np.pi > 0)+